adamic_adar_index raises error, division by 0 error.

Jaccard always gets 0.

There is pairs such as ('gaming', 'askreddit'), ('askreddit', 'gaming').

There are more functions that can incorporate community information if we want.  

In [ ]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
G_train = nx.convert_matrix.from_pandas_edgelist(df_train, source="SOURCE_SUBREDDIT", target="TARGET_SUBREDDIT", edge_attr="LINK_SENTIMENT")
G_test = nx.convert_matrix.from_pandas_edgelist(df_test, source="SOURCE_SUBREDDIT", target="TARGET_SUBREDDIT", edge_attr="LINK_SENTIMENT")
test_set = set(G_test.edges)

In [ ]:
def common_neighbor(G, size):
    all_node_pairs = set(itertools.product(G.nodes, repeat = 2))
    all_unconnected_node_pairs = all_node_pairs - set(G.edges)
    score = {}
    for node_pair in all_unconnected_node_pairs:
        if node_pair[0] != node_pair[1]:
            score[node_pair] = len(set(G.neighbors(node_pair[0])) & set(G.neighbors(node_pair[1])))
    sorted_guesses = sorted(score.items(), key = lambda x: x[1], reverse = True)
    sorted_best_guesses = sorted_guesses[:size]
    return set([guess[0] for guess in sorted_best_guesses])

def jaccard(G, size):
    all_node_pairs = set(itertools.product(G.nodes, repeat = 2))
    all_unconnected_node_pairs = all_node_pairs - set(G.edges)
    j = list(nx.jaccard_coefficient(G_train, ebunch=all_unconnected_node_pairs))
    score = {}
    for node_pair in j:
        if node_pair[0] != node_pair[1]:
            score[(node_pair[0],node_pair[1])] = node_pair[2]
    sorted_guesses = sorted(score.items(), key = lambda x: x[1], reverse = True)
    sorted_best_guesses = sorted_guesses[:size]
    return set([guess[0] for guess in sorted_best_guesses])

def pa(G, size):
    all_node_pairs = set(itertools.product(G.nodes, repeat = 2))
    all_unconnected_node_pairs = all_node_pairs - set(G.edges)
    pa = list(nx.preferential_attachment(G_train, ebunch=all_unconnected_node_pairs))
    score = {}
    for node_pair in pa:
        if node_pair[0] != node_pair[1]:
            score[(node_pair[0],node_pair[1])] = node_pair[2]
    sorted_guesses = sorted(score.items(), key = lambda x: x[1], reverse = True)
    sorted_best_guesses = sorted_guesses[:size]
    return set([guess[0] for guess in sorted_best_guesses])

def rai(G, size):
    all_node_pairs = set(itertools.product(G.nodes, repeat = 2))
    all_unconnected_node_pairs = all_node_pairs - set(G.edges)
    rai = list(nx.resource_allocation_index(G_train, ebunch=all_unconnected_node_pairs))
    score = {}
    for node_pair in rai:
        if node_pair[0] != node_pair[1]:
            score[(node_pair[0],node_pair[1])] = node_pair[2]
    sorted_guesses = sorted(score.items(), key = lambda x: x[1], reverse = True)
    sorted_best_guesses = sorted_guesses[:size*2:2]
    return set([guess[0] for guess in sorted_best_guesses])

def aai(G, size):
    all_node_pairs = set(itertools.product(G.nodes, repeat = 2))
    all_unconnected_node_pairs = all_node_pairs - set(G.edges)
    aai = list(nx.adamic_adar_index(G_train, ebunch=all_unconnected_node_pairs))
    score = {}
    for node_pair in aai:
        if node_pair[0] != node_pair[1]:
            score[(node_pair[0],node_pair[1])] = node_pair[2]
    sorted_guesses = sorted(score.items(), key = lambda x: x[1], reverse = True)
    sorted_best_guesses = sorted_guesses[:size]
    return set([guess[0] for guess in sorted_best_guesses])

def random_guess(G, size):
    all_node_pairs = set(itertools.product(G.nodes, repeat = 2))
    all_unconnected_node_pairs = all_node_pairs - set(G.edges)
    return set(random.sample(all_unconnected_node_pairs, size))

In [ ]:
def score(G, func, size):
    guesses = func(G, size)
    precision = len(guesses & test_set) / len(guesses)
    recall = len(guesses & test_set) / len(test_set)
    return (precision, recall)

In [ ]:
funcs = [common_neighbor, jaccard, pa, rai, random_guess]
func_labels = ['common neighbor', 'jaccard', 'preferential attachment', 'resource allocation index', 'random guess']
for label,func in zip(func_labels, funcs):
    print(label, score(G_train, func, 1000))